In [18]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings(action='ignore')
import pickle
import time
from sklearn.ensemble import GradientBoostingClassifier 
from scipy.stats import randint
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization
from xgboost import XGBClassifier # model 
from xgboost import plot_importance
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [14]:
import sys   
!{sys.executable} -m pip install lightgbm

In [2]:
def seed_everything(seed):
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
    os.environ['PYTHONHASHSEED'] = str(seed) 
    np.random.seed(seed) #넘파이를 사용할 경우
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
    
seed_everything(37) # Seed 고정

# Data Load

In [5]:
#데이터프레임 불러오기
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [3]:
subXGB_b = pd.read_csv('./sample_submission.csv')
subl_GBM_b = pd.read_csv('./sample_submission.csv')
subRF_b = pd.read_csv('./sample_submission.csv')
sub_v = pd.read_csv('./sample_submission.csv')

# Data Preprocessing

데이터 전처리(Data preprocessing)의 목적은 

주어진원본 데이터를 신경망에 적용하기 쉽도록 만드는 것이다.

벡터화(vectorization), 정규화(normalization), 

특성 추출(Feature Engineering)등이 포함된다.

In [6]:
train_df = train_df.fillna(-100)
test_df = test_df.fillna(-100)

### Feature Selection

outlier 이상체 제거한 최종 데이터 프레임 저장하기 & 모든 value가 결측치이거나 0.0인 행 지운 test data도 저장하기

모델링을 하기 앞서, 독립변수 X와 종속변수 y를 설정해야한다.

**학습에 사용할 변수 X와 예측할 변수 y를 분리**

In [7]:
train_y = train_df['Y_Class']

학습에 쓰이지 않을 column들을 제거한다.

In [8]:
train_x = train_df.drop(columns = ['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
#모델 학습이 끝나고 예측에 쓰일 test데이터
test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

범주형 데이터를 수치 데이터로 전환하기 위해 LabelEncoder 활용

In [9]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i]) #원래 column 값을 기준으로 fit.
    train_x[i] = le.transform(train_x[i]) #수치화, 수치로 변형
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


### Modeling

# Bayesian Optimization

### RandomForest bayesian

In [10]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

start = time.time()
rf_parameter_bounds = {
                      'n_estimators' : (30,100),
                      'max_depth' : (1,3) # 나무의 깊이
#                       'min_samples_split' : (2, 10),
#                       'min_samples_leaf' : (1,17)
                      }

def RF_bo(n_estimators, max_depth):

    rf_params = {
              'n_estimators' : int(round(n_estimators)),
              'max_depth' : int(round(max_depth))  
              }

    rf_b = RandomForestClassifier(**rf_params)
    train_xx, valid_x, train_yy, valid_y = train_test_split(train_x, train_y, test_size = 0.2)
    rf_b.fit(train_xx,train_yy)

    score = f1_score(valid_y, rf_b.predict(valid_x), average = 'macro')

    return score


BO_rf = BayesianOptimization(f = RF_bo, pbounds = rf_parameter_bounds, random_state = 37)
 
BO_rf.maximize(init_points = 5, n_iter = 5)
end = time.time()
print('time elapsed:', end - start)

|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
| 1         | 0.4425    | 2.889     | 62.49     |
| 2         | 0.2875    | 1.386     | 70.73     |
| 3         | 0.3129    | 2.24      | 77.9      |
| 4         | 0.2764    | 1.207     | 82.18     |
| 5         | 0.3243    | 1.564     | 82.74     |
| 6         | 0.4153    | 2.532     | 61.92     |
| 7         | 0.4845    | 2.076     | 63.51     |
| 8         | 0.4416    | 2.938     | 65.42     |
| 9         | 0.3574    | 1.048     | 65.04     |
| 10        | 0.4905    | 2.962     | 64.01     |
time elapsed: 2.080186128616333


In [11]:
#macro_f1_score
rf_tune_b = RandomForestClassifier(random_state = 37, 
                                   max_depth = 3, 
                                   n_estimators = 64)
start = time.time()
rf_tune_b.fit(train_x,train_y)
preds = rf_tune_b.predict(test_x)
end = time.time()
print('time elapsed:', end - start)
subRF_b['Y_Class'] == preds
subRF_b.to_csv('./t16_RandomForest_bayesian.csv', index = False)

time elapsed: 0.2179419994354248


### lightGBM bayesian

In [22]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

start = time.time()
lgbm_parameter_bounds = {

                      'n_estimators' : (30,100),

                      'max_depth' : (1,3), # 나무의 깊이

                      'subsample' : (0.5,1)

                      }

def lgbm_bo(n_estimators,max_depth, subsample):

    lgbm_params = {

              'n_estimators' : int(round(n_estimators)),

              'max_depth' : int(round(max_depth)),

               'subsample' : int(round(subsample)),      

              }

    lgbm = LGBMClassifier(**lgbm_params) 
    train_xx, valid_x, train_yy, valid_y = train_test_split(train_x, train_y, test_size = 0.2)
    lgbm.fit(train_xx,train_yy)

    score = f1_score(valid_y, lgbm.predict(valid_x), average = 'macro')

    return score


BO_lgbm = BayesianOptimization(f = lgbm_bo, pbounds = lgbm_parameter_bounds, random_state = 37)
 
BO_lgbm.maximize(init_points = 5, n_iter = 5)
end = time.time()
print('time elapsed:', end - start)

|   iter    |  target   | max_depth | n_esti... | subsample |
-------------------------------------------------------------
| 1         | 0.5601    | 2.889     | 62.49     | 0.5964    |
| 2         | 0.578     | 2.164     | 73.41     | 0.8421    |
| 3         | 0.4813    | 1.207     | 82.18     | 0.641     |
| 4         | 0.6742    | 2.507     | 85.49     | 0.8137    |
| 5         | 0.7006    | 1.887     | 97.44     | 0.9485    |
| 6         | 0.5741    | 2.972     | 33.21     | 0.8043    |
| 7         | 0.5659    | 2.837     | 59.74     | 0.6364    |
| 8         | 0.6848    | 2.059     | 97.85     | 0.6132    |
| 9         | 0.6302    | 2.809     | 95.78     | 0.5498    |
| 10        | 0.6456    | 3.0       | 87.5      | 0.9208    |
time elapsed: 7.275197267532349


In [23]:
#macro_f1_score
lgbm_tune_b = LGBMClassifier(random_state = 37, 
                                   max_depth = 2, 
                                   n_estimators = 97,
                                    subsample = 0.9485)
start = time.time()
lgbm_tune_b.fit(train_x,train_y)
preds = lgbm_tune_b.predict(test_x)
end = time.time()
print('time elapsed:', end - start)
subl_GBM_b['Y_Class'] == preds
subl_GBM_b.to_csv('./t16_LGBM_bayesian.csv', index = False)

time elapsed: 0.8395681381225586


### XGBoost bayesian

In [24]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

start = time.time()
xgb_parameter_bounds = {

                      'gamma' : (0,10),

                      'max_depth' : (1,3), # 나무의 깊이

                      'subsample' : (0.5,1)

                      }

def xgb_bo(gamma, max_depth, subsample):

    xgb_params = {

              'gamma' : int(round(gamma)),

              'max_depth' : int(round(max_depth)),

               'subsample' : int(round(subsample)),      

              }

    xgb = XGBClassifier(**xgb_params) 
    train_xx, valid_x, train_yy, valid_y = train_test_split(train_x, train_y, test_size = 0.2)
    xgb.fit(train_xx,train_yy)

    score = f1_score(valid_y, xgb.predict(valid_x), average = 'macro')

    return score


BO_xgb = BayesianOptimization(f = xgb_bo, pbounds = xgb_parameter_bounds, random_state = 37)
 
BO_xgb.maximize(init_points = 5, n_iter = 5)
end = time.time()
print('time elapsed:', end - start)

|   iter    |  target   |   gamma   | max_depth | subsample |
-------------------------------------------------------------
| 1         | 0.3496    | 9.445     | 1.928     | 0.5964    |
| 2         | 0.415     | 5.819     | 2.24      | 0.8421    |
| 3         | 0.539     | 1.034     | 2.491     | 0.641     |
| 4         | 0.3872    | 7.534     | 2.585     | 0.8137    |
| 5         | 0.5651    | 4.435     | 2.927     | 0.9485    |
| 6         | 0.5662    | 2.848     | 3.0       | 1.0       |
| 7         | 0.09113   | 2.789     | 1.0       | 0.5       |
| 8         | 0.6179    | 3.599     | 2.976     | 0.739     |
| 9         | 0.6819    | 0.0       | 3.0       | 1.0       |
| 10        | 0.6979    | 0.0       | 1.455     | 1.0       |
time elapsed: 43.56435179710388


In [29]:
#macro_f1_score
xgb_tune_b = XGBClassifier(random_state = 37,
                                   gamma = 0.0,
                                   max_depth = 3, 
                                    subsample = 1.0)
start = time.time()
xgb_tune_b.fit(train_x,train_y)
preds = xgb_tune_b.predict(test_x)
end = time.time()
print('time elapsed:', end - start)
subXGB_b['Y_Class'] == preds
subXGB_b.to_csv('./t16_XGB_bayesian.csv', index = False)

time elapsed: 6.27745509147644


### VOTING

In [39]:
LGBM = LGBMClassifier(max_depth = 2, n_estimators=97, subsample = 0.9485)

XGB = XGBClassifier(gamma = 0.0, max_depth = 3, subsample = 1.0)

RF = RandomForestClassifier(max_depth = 3, n_estimators = 64)

In [40]:
VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [41]:
VC.fit(train_x,train_y)
preds = VC.predict(test_x)
sub_v['Y_Class'] == preds
sub_v.to_csv('./t16_RFXGBLGBM_bayesian_voting.csv', index = False)

In [44]:
subGBM_r = pd.read_csv('./t10_GBM_RandomSearchCV_niter50.csv')

subXGB_b = pd.read_csv('./sample_submission.csv')
subl_GBM_b = pd.read_csv('./sample_submission.csv')
subRF_b = pd.read_csv('./sample_submission.csv')
sub_v = pd.read_csv('./sample_submission.csv')

In [45]:
sub = pd.concat([subGBM_r, subXGB_b, subl_GBM_b, subRF_b, sub_v], axis = 1)
sub

,PRODUCT_ID,Y_Class,PRODUCT_ID,Y_Class,PRODUCT_ID,Y_Class,PRODUCT_ID,Y_Class,PRODUCT_ID,Y_Class
0,TEST_000,1,TEST_000,0,TEST_000,0,TEST_000,0,TEST_000,0
1,TEST_001,1,TEST_001,0,TEST_001,0,TEST_001,0,TEST_001,0
2,TEST_002,1,TEST_002,0,TEST_002,0,TEST_002,0,TEST_002,0
3,TEST_003,1,TEST_003,0,TEST_003,0,TEST_003,0,TEST_003,0
4,TEST_004,0,TEST_004,0,TEST_004,0,TEST_004,0,TEST_004,0
...,...,...,...,...,...,...,...,...,...,...
305,TEST_305,2,TEST_305,0,TEST_305,0,TEST_305,0,TEST_305,0
306,TEST_306,2,TEST_306,0,TEST_306,0,TEST_306,0,TEST_306,0
307,TEST_307,1,TEST_307,0,TEST_307,0,TEST_307,0,TEST_307,0
308,TEST_308,2,TEST_308,0,TEST_308,0,TEST_308,0,TEST_308,0


In [47]:
subl_GBM_b.describe()

,Y_Class
count,310.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


# XGBoost randomsearch

In [98]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import time

param_distribs = {
    'min_child_weight': randint(low = 1, high = 10),
    'gamma': randint(low = 1, high = 10),
    'max_depth': randint(low = 5, high = 10),
    'lambda' : randint(low = 1, high = 10),
    'alpha' : randint(low = 1, high = 10)
}
start = time.time()
xgboost_r = XGBClassifier(seed=37).fit(train_x,train_y)
rand_cv = RandomizedSearchCV(xgboost_r, 
                            param_distributions=param_distribs,
                            cv = 5, 
                            n_iter = 50,
                            scoring = 'f1_macro',
                            n_jobs = -1,
                            verbose=3)
rand_cv.fit(train_x, train_y)
preds = rand_cv.predict(test_x)
end = time.time()
subXGB_r['Y_Class'] = preds
subXGB_r.to_csv('./t15_XGBoost_RandomSearchCV_niter50_dpo4-100.csv', index=False)

print(f'최적 하이퍼 파라미터: {rand_cv.best_params_}')
print(f'최고 예측 정확도: {(rand_cv.best_score_)*100:.4f}')
print(f'time elapsed: {end-start}')

Fitting 5 folds for each of 50 candidates, totalling 250 fits
최적 하이퍼 파라미터: {'alpha': 4, 'colsample_bytree': 0, 'eta': 0, 'gamma': 1, 'lambda': 3, 'max_depth': 6, 'min_child_weight': 8}
최고 예측 정확도: 44.3738
time elapsed: 98.70302700996399


# voting classifier

In [108]:
eclf = VotingClassifier(
    estimators=[('GBM', gbm_clf), ('XGB', xgboost_r)], voting='soft')
    
params = {'GBM__n_estimators': [1, 200], 'GBM__min_samples_split': [1, 10], 
              'GBM__min_samples_leaf': [1, 17],'XGB__max_depth': [0, 10], 
              'XGB__alpha':[0,5], 'XGB__lambda':[0,5],
              'XGB__min_child_weight':[5,8] }

ran = RandomizedSearchCV(estimator=eclf, 
                         param_distributions = params, 
                         n_iter = 50, 
                         cv=5, 
                         n_jobs = -1, 
                         scoring = 'f1_macro',
                         verbose = 3)
ran.fit(train_x, train_y)
preds = ran.predict(test_x)
sub_voting_r['Y_Class'] = preds
sub_voting_r.to_csv('./t15_GBMXGBVotingsoft_RandomSearchCV_niter50_dpo4-100.csv', index=False)
print(f'최적 하이퍼 파라미터: {rand_cv.best_params_}')
print(f'최고 예측 정확도: {(rand_cv.best_score_)*100:.4f}')
print(f'time elapsed: {end-start}')

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 4/5] END GBM__min_samples_leaf=1, GBM__min_samples_split=1, GBM__n_estimators=1, XGB__alpha=0, XGB__lambda=0, XGB__max_depth=0, XGB__min_child_weight=8;, score=nan total time=   0.1s
[CV 4/5] END GBM__min_samples_leaf=1, GBM__min_samples_split=10, GBM__n_estimators=1, XGB__alpha=0, XGB__lambda=0, XGB__max_depth=10, XGB__min_child_weight=5;, score=0.268 total time=  38.9s
[CV 5/5] END GBM__min_samples_leaf=17, GBM__min_samples_split=1, GBM__n_estimators=1, XGB__alpha=5, XGB__lambda=0, XGB__max_depth=0, XGB__min_child_weight=8;, score=nan total time=   0.1s
[CV 3/5] END GBM__min_samples_leaf=1, GBM__min_samples_split=10, GBM__n_estimators=1, XGB__alpha=0, XGB__lambda=5, XGB__max_depth=0, XGB__min_child_weight=5;, score=nan total time=   0.8s
[CV 5/5] END GBM__min_samples_leaf=1, GBM__min_samples_split=10, GBM__n_estimators=1, XGB__alpha=0, XGB__lambda=5, XGB__max_depth=0, XGB__min_child_weight=5;, score=nan total time=   0

[CV 3/5] END GBM__min_samples_leaf=1, GBM__min_samples_split=1, GBM__n_estimators=1, XGB__alpha=0, XGB__lambda=0, XGB__max_depth=0, XGB__min_child_weight=8;, score=nan total time=   0.1s
[CV 3/5] END GBM__min_samples_leaf=17, GBM__min_samples_split=1, GBM__n_estimators=200, XGB__alpha=0, XGB__lambda=5, XGB__max_depth=10, XGB__min_child_weight=5;, score=nan total time=   0.1s
[CV 1/5] END GBM__min_samples_leaf=17, GBM__min_samples_split=10, GBM__n_estimators=200, XGB__alpha=5, XGB__lambda=5, XGB__max_depth=0, XGB__min_child_weight=5;, score=nan total time= 1.0min
[CV 4/5] END GBM__min_samples_leaf=17, GBM__min_samples_split=10, GBM__n_estimators=200, XGB__alpha=0, XGB__lambda=5, XGB__max_depth=0, XGB__min_child_weight=8;, score=nan total time= 1.0min
[CV 2/5] END GBM__min_samples_leaf=1, GBM__min_samples_split=10, GBM__n_estimators=200, XGB__alpha=5, XGB__lambda=5, XGB__max_depth=10, XGB__min_child_weight=5;, score=0.446 total time= 1.7min
[CV 5/5] END GBM__min_samples_leaf=17, GBM__min

[CV 1/5] END GBM__min_samples_leaf=1, GBM__min_samples_split=1, GBM__n_estimators=1, XGB__alpha=0, XGB__lambda=0, XGB__max_depth=0, XGB__min_child_weight=8;, score=nan total time=   0.1s
[CV 5/5] END GBM__min_samples_leaf=1, GBM__min_samples_split=10, GBM__n_estimators=1, XGB__alpha=0, XGB__lambda=0, XGB__max_depth=10, XGB__min_child_weight=5;, score=0.298 total time=  38.5s
[CV 5/5] END GBM__min_samples_leaf=17, GBM__min_samples_split=10, GBM__n_estimators=200, XGB__alpha=5, XGB__lambda=5, XGB__max_depth=0, XGB__min_child_weight=5;, score=nan total time= 1.1min
[CV 4/5] END GBM__min_samples_leaf=1, GBM__min_samples_split=10, GBM__n_estimators=200, XGB__alpha=0, XGB__lambda=0, XGB__max_depth=10, XGB__min_child_weight=5;, score=0.320 total time= 1.7min
[CV 4/5] END GBM__min_samples_leaf=17, GBM__min_samples_split=10, GBM__n_estimators=1, XGB__alpha=0, XGB__lambda=0, XGB__max_depth=0, XGB__min_child_weight=8;, score=nan total time=   0.7s
[CV 3/5] END GBM__min_samples_leaf=17, GBM__min_s

[CV 1/5] END GBM__min_samples_leaf=1, GBM__min_samples_split=10, GBM__n_estimators=1, XGB__alpha=0, XGB__lambda=0, XGB__max_depth=10, XGB__min_child_weight=5;, score=0.341 total time=  38.7s
[CV 1/5] END GBM__min_samples_leaf=17, GBM__min_samples_split=1, GBM__n_estimators=1, XGB__alpha=5, XGB__lambda=0, XGB__max_depth=0, XGB__min_child_weight=8;, score=nan total time=   0.1s
[CV 2/5] END GBM__min_samples_leaf=17, GBM__min_samples_split=1, GBM__n_estimators=1, XGB__alpha=5, XGB__lambda=0, XGB__max_depth=0, XGB__min_child_weight=8;, score=nan total time=   0.1s
[CV 3/5] END GBM__min_samples_leaf=17, GBM__min_samples_split=1, GBM__n_estimators=1, XGB__alpha=5, XGB__lambda=0, XGB__max_depth=0, XGB__min_child_weight=8;, score=nan total time=   0.1s
[CV 4/5] END GBM__min_samples_leaf=17, GBM__min_samples_split=1, GBM__n_estimators=1, XGB__alpha=5, XGB__lambda=0, XGB__max_depth=0, XGB__min_child_weight=8;, score=nan total time=   0.1s
[CV 2/5] END GBM__min_samples_leaf=1, GBM__min_samples_sp

## RandomForest 파라미터 최적화

랜덤포레스트 파라미터 최적화 x default

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

train_xx, valid_x, train_yy, valid_y = train_test_split(train_x, train_y, test_size = 0.2)

rf= RandomForestClassifier(random_state=37).fit(train_xx, train_yy)

score = f1_score(valid_y, rf.predict(valid_x), average = 'macro')
print(score)
rf= RandomForestClassifier(random_state=37).fit(train_x, train_y)
preds = rf.predict(test_x)
submission_d['Y_Class'] = preds
submission_d.to_csv('./t13_Randomforest_default.csv', index = False)

0.9532679738562092


In [22]:
submission_d.describe()

,Y_Class
count,310.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


### GridSearchCV

In [625]:
from sklearn.model_selection import GridSearchCV

#랜덤 포레스트 하이퍼 파라미터 튜닝 
params = {'max_depth':[None, 2,3,4,5,6,7,8],
#           'max_feature': [], 두 가지 어떻게 조절할지 모르겠음
#           'max_leaf_nodes' :[]
       	  'min_samples_leaf':[1,2,3,16,17,18], 
          'min_samples_split':[1,2,3,4,8],
         'n_estimators':[110,120,125,128,130,132,135]}
start = time.time()        
#랜덤 포래스트 객체 생성 후 gridsearchcv수행
rf_g = RandomForestClassifier(random_state = 37, n_jobs = -1)
grid_cv = GridSearchCV(rf_g, param_grid = params, cv = 2, n_jobs = -1)
grid_cv.fit(train_x, train_y)
preds = grid_cv.predict(test_x)

#gridsearchcv를 이용해 최적으로 학습된 estimator로 예측 수행
gb_pred = grid_cv.best_estimator_
preds = gb_pred.predict(test_x)
end = time.time()
print('time elapsed:', end - start)
print('최적 하이퍼 파라미터: \n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))
submission_g['Y_Class'] = preds
submission_g.to_csv('./t13_RandomForest_grid.csv', index=False)

time elapsed: 290.8840320110321
최적 하이퍼 파라미터: 
 {'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 110}
최고 예측 정확도: 0.6940


### RandomSearchCV

In [626]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
    'n_estimators' : randint(low=1, high=200),
    'min_samples_split': randint(low = 1, high = 10),
    'min_samples_leaf': randint(low = 1, high = 17),
    'max_depth': randint(low = 1, high = 8)
}
start = time.time()
rf_r = RandomForestClassifier(random_state=37)
rand_cv = RandomizedSearchCV(rf_r, 
                            param_distributions=param_distribs,
                            cv = 5, 
                            n_jobs = -1,
                            verbose=3)
rand_cv.fit(train_x, train_y)
preds = rand_cv.predict(test_x)
end = time.time()
print('time elapsed:', end - start)
print(f'최적 하이퍼 파라미터: {rand_cv.best_params_}')
print(f'최고 예측 정확도: {(rand_cv.best_score_)*100:.4f}')
submission_r['Y_Class'] = preds
submission_r.to_csv('./t13_RandomForest_random.csv', index=False)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
time elapsed: 4.2614710330963135
최적 하이퍼 파라미터: {'max_depth': 7, 'min_samples_leaf': 8, 'min_samples_split': 5, 'n_estimators': 25}
최고 예측 정확도: 86.7521
[CV 5/5] END max_depth=1, min_samples_leaf=16, min_samples_split=7, n_estimators=192;, score=0.681 total time=   0.7s
[CV 3/5] END max_depth=2, min_samples_leaf=8, min_samples_split=5, n_estimators=166;, score=0.983 total time=   0.9s
[CV 5/5] END max_depth=4, min_samples_leaf=7, min_samples_split=5, n_estimators=88;, score=0.681 total time=   0.6s
[CV 5/5] END max_depth=1, min_samples_leaf=7, min_samples_split=9, n_estimators=82;, score=0.681 total time=   0.5s
[CV 2/5] END max_depth=5, min_samples_leaf=4, min_samples_split=7, n_estimators=17;, score=0.950 total time=   0.2s
[CV 1/5] END max_depth=4, min_samples_leaf=3, min_samples_split=7, n_estimators=50;, score=0.883 total time=   0.4s
[CV 3/5] END max_depth=1, min_samples_leaf=16, min_samples_split=7, n_estimators=192;, scor

In [ ]:
# #GBM Gradient Boosting Machine
# from sklearn.ensemble import GradientBoostingClassifier 

# # 예시 데이터셋 불러오기
# clf = GradientBoostingClassifier(random_state=37)
# clf.fit(train_x, train_y.values)
# preds = gb_clf.predict(test_x)
# submission['Y_Class'] = preds
# submission.to_csv('./t11_GBM.csv', index=False)

In [18]:
# #train_df_0 index 재설정
# train_df_0 = train_df_0.reset_index(inplace = False, drop = True)
# train_df_0

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_022,0,0.517719,2022-06-14 8:53,T100304,T_31,2.0,102.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAIN_023,0,0.519090,2022-06-14 9:01,T100304,T_31,2.0,102.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAIN_028,0,0.521249,2022-06-19 20:26,T010305,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAIN_034,0,0.521524,2022-06-21 17:36,T050304,A_31,NaN,NaN,NaN,NaN,...,55.03,52.24,55.33,57.49,67.31,1.0,NaN,NaN,NaN,NaN
4,TRAIN_066,0,0.524408,2022-06-25 21:38,T010305,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,TRAIN_583,0,0.522340,2022-09-05 8:34,T050304,A_31,NaN,NaN,NaN,NaN,...,51.71,59.64,54.61,57.05,63.18,1.0,NaN,NaN,NaN,NaN
84,TRAIN_584,0,0.519519,2022-09-05 11:09,T010305,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,TRAIN_585,0,0.515214,2022-09-05 11:17,T010306,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,TRAIN_594,0,0.524022,2022-09-08 22:38,T050304,A_31,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
